# Importing Modules

In [3]:
# Usual imports
import numpy as np
import pandas as pd
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os
#print(os.listdir("../input"))

# Plotly based imports for visualization
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

# spaCy based imports
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
!python -m spacy download en_core_web_lg


    Linking successful
    /Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/en_core_web_lg
    -->
    /Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')



# Creating a spaCy object

In [4]:
nlp = spacy.load('en_core_web_lg')

# Importing Data

In [6]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


# Filtering for Only 2018 

In [19]:
data18 = data.loc[data['Year'] =='2018']
data18

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [15]:
doc = nlp(data["content"][5])
spacy.displacy.render(doc, style='ent',jupyter=True)

In [16]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

# Lemmatization

In [21]:
review = str(" ".join([i.lemma_ for i in doc]))

In [22]:
review

"“ will -PRON- shut the fuck up and listen ? ” , ask the crowd of chatty teenybopper gather at the whisky a go go in los angele in february 1969 . the kid answer -PRON- question with more talk . the unceremonious , promotional tour be unfold with the kind of due - pay , shit - eat milestone that the belfast singer have sadly become accustomed to in the previous few year . sure , some sign of positive change be afoot : warner bro . have successfully purchase -PRON- bang record contract from mob - up charlatan on the edge of the music industry , and -PRON- surprising acoustic rebirth in , have be breathlessly notice by some critic , even if -PRON- could barely find a copy in a record store . but nine - minute , three - chord jam back by jazz player do not often become radio hit , and morrison know that what -PRON- now need be to generate material that would be play over the airwave . as -PRON- piano player at the time jef lab note , “ i think warner have pretty much tell -PRON- , ‘ -PRON

In [23]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

# POS Tagging

In [24]:
for i in nlp(review):
    print(i,"=>",i.pos_)

“ => ADV
will => VERB
-PRON- => PUNCT
shut => VERB
the => DET
fuck => NOUN
up => PART
and => CCONJ
listen => VERB
? => PUNCT
” => INTJ
, => PUNCT
ask => VERB
the => DET
crowd => NOUN
of => ADP
chatty => ADJ
teenybopper => NOUN
gather => VERB
at => ADP
the => DET
whisky => NOUN
a => DET
go => NOUN
go => VERB
in => ADP
los => PROPN
angele => NOUN
in => ADP
february => ADJ
1969 => NUM
. => PUNCT
the => DET
kid => NOUN
answer => NOUN
-PRON- => NUM
question => NOUN
with => ADP
more => ADJ
talk => NOUN
. => PUNCT
the => DET
unceremonious => ADJ
, => PUNCT
promotional => ADJ
tour => NOUN
be => VERB
unfold => ADJ
with => ADP
the => DET
kind => NOUN
of => ADP
due => ADJ
- => PUNCT
pay => NOUN
, => PUNCT
shit => NOUN
- => PUNCT
eat => NOUN
milestone => NOUN
that => ADP
the => DET
belfast => NOUN
singer => NOUN
have => VERB
sadly => ADV
become => VERB
accustomed => ADJ
to => ADP
in => ADP
the => DET
previous => ADJ
few => ADJ
year => NOUN
. => PUNCT
sure => ADJ
, => PUNCT
some => DET
sign => NOUN

’s => PART
revisionist => ADJ
history => NOUN
in => ADP
regard => NOUN
to => ADP
this => DET
artistic => ADJ
collaboration => NOUN
would => VERB
reach => VERB
-PRON- => NUM
apex => NOUN
in => ADP
2009 => NUM
when => ADV
-PRON- => PUNCT
glibly => ADV
tell => VERB
radio => NOUN
host => NOUN
don => NOUN
imus => NOUN
that => ADP
all => DET
merenstein => NOUN
do => VERB
during => ADP
, => PUNCT
be => VERB
fetch => VERB
the => DET
musician => NOUN
sandwich => NOUN
, => PUNCT
but => CCONJ
in => ADP
1969 => NUM
, => PUNCT
morrison => NOUN
’s => PART
rejection => NOUN
of => ADP
-PRON- => ADJ
producer => NOUN
would => VERB
be => VERB
protract => VERB
and => CCONJ
painful => ADJ
. => PUNCT
session => NOUN
for => ADP
, => PUNCT
begin => VERB
at => ADP
century => NOUN
sound => NOUN
studio => NOUN
with => ADP
most => ADJ
of => ADP
the => DET
musician => NOUN
from => ADP
, => PUNCT
and => CCONJ
engineer => NOUN
brook => NOUN
arthur => PROPN
back => ADV
on => ADP
the => DET
board => NOUN
as => ADV
wel

suddenly => ADV
become => VERB
such => ADJ
an => DET
accessible => ADJ
lyrical => ADJ
touchstone => NOUN
for => ADP
the => DET
songwriter => NOUN
. => PUNCT
one => NUM
of => ADP
these => DET
transient => ADJ
- => PUNCT
song => NOUN
, => PUNCT
“ => ADJ
into => ADP
the => DET
mystic => ADJ
” => NOUN
function => NOUN
as => ADP
the => DET
album => NOUN
’s => PART
moving => ADJ
- => PUNCT
spiritual => ADJ
- => PUNCT
quest => NOUN
- => PUNCT
centerpiece => NOUN
, => PUNCT
just => ADV
as => ADP
“ => ADJ
madame => NOUN
george => NOUN
” => NOUN
do => VERB
on => ADV
, => PUNCT
. => PUNCT
“ => ADV
into => ADP
the => DET
mystic => ADJ
” => NOUN
be => VERB
such => ADJ
a => DET
soothing => ADJ
and => CCONJ
reassuring => ADJ
track => NOUN
that => ADP
a => DET
poll => ADJ
conduct => NOUN
by => ADP
the => DET
bbc => NOUN
in => ADP
the => DET
late => ADJ
’90 => NOUN
find => VERB
that => DET
-PRON- => PUNCT
have => VERB
become => VERB
— => PUNCT
no => DET
joke => NOUN
— => PUNCT
a => DET
popular => ADJ
s

to => PART
become => VERB
famous.,“make => ADJ
dream => NOUN
come => VERB
true => ADJ
if => ADP
-PRON- => PROPN
want => VERB
-PRON- => PUNCT
to => PART
, => PUNCT
” => NOUN
morrison => NOUN
sing => VERB
on => ADP
the => DET
album => NOUN
’s => PART
penultimate => ADJ
track => NOUN
, => PUNCT
“ => ADJ
everyone => NOUN
. => PUNCT
” => NOUN
for => ADP
the => DET
singer => NOUN
from => ADP
belfast => PROPN
, => PUNCT
the => DET
long => ADJ
delay => NOUN
dream => NOUN
- => PUNCT
come => VERB
- => PUNCT
true => ADJ
would => VERB
become => VERB
an => DET
ongoing => ADJ
catch-22 => NOUN
. => PUNCT
finally => ADV
, => PUNCT
morrison => NOUN
could => VERB
provide => VERB
for => ADP
-PRON- => PUNCT
new => ADJ
family => NOUN
and => CCONJ
create => VERB
the => DET
precise => ADJ
kind => NOUN
of => ADP
music => NOUN
-PRON- => PUNCT
wish => NOUN
to => PART
make => VERB
, => PUNCT
but => CCONJ
there => ADV
be => VERB
a => DET
downside => NOUN
to => ADP
the => DET
success => NOUN
. => PUNCT
“ => ADV
be

In [25]:
# Parser for reviews
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [26]:
tqdm.pandas()
data18["processed_content"] = data18["content"].progress_apply(spacy_tokenizer)

100%|██████████| 1198/1198 [00:15<00:00, 75.38it/s]


In [68]:
# Creating a vectorizer
vectorizer = CountVectorizer(min_df=2, max_df=0.80, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data18["processed_content"])

In [69]:
type(data_vectorized)

scipy.sparse.csr.csr_matrix

In [70]:
NUM_TOPICS = 10

In [71]:
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [72]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 

In [73]:
# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

In [74]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [75]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('nielson', 16.40677908347809), ('tracy', 16.26525814429546), ('henson', 10.278263108480907), ('mumford', 9.632427394469868), ('orcutt', 9.496350712020735), ('greta', 8.735719690391575), ('oldham', 7.873425773511127), ('umo', 7.666812777208457), ('gill', 6.988552649452848), ('fleet', 6.608795636857422)]
Topic 1:
[('lavigne', 26.29895187024544), ('plunkett', 13.810496930272087), ('zhaan', 8.56018056057902), ('tabla', 6.071025727556031), ('yuzawa', 2.702501945508423), ('hironori', 1.7951035267333997), ('rourke', 1.0092153450017327), ('cagily', 0.8813500331522424), ('aerosmith', 0.8168270847117076), ('music', 0.1263566448065978)]
Topic 2:
[('earl', 50.287401584803185), ('tyler', 26.037494814727523), ('algernon', 25.07363132936507), ('crutchfield', 21.78992664182593), ('yachty', 18.36218478077099), ('albarn', 18.257307989600005), ('dacus', 16.49447351495177), ('dunn', 15.891703651184923), ('korean', 15.107087663532258), ('coral', 14.843628711774205)]
Topic 3:
[('carey'

In [60]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      -16.001371  -37.923500       1        1  97.184703
3     -131.814667   10.716189       2        1   1.823064
2       96.377037  -95.926277       3        1   0.198938
1     -130.493179  122.173157       4        1   0.182488
8     -125.827118 -100.587883       5        1   0.165264
7      -13.590483 -152.292938       6        1   0.130844
0       91.711349  126.822174       7        1   0.120523
5      -18.139288   64.165199       8        1   0.100032
9      -20.520655  178.535339       9        1   0.048471
6       97.659538   15.523580      10        1   0.045673, topic_info=      Category         Freq           Term        Total  loglift  logprob
1130   Default  1776.000000           band  1776.000000  30.0000  30.0000
11752  Default  2202.000000         record  2202.000000  29.0000  29.0000
12325  Default  1081.000000           rock  1081.000000  28.0000  28.0000
13464  Default  3906.000000           song  3906.000000  27.0000  27.0000
6379   Default  1136.000000         guitar  1136.000000  26.0000  26.0000
393    Default  4280.000000          album  4280.000000  25.0000  25.0000
14720  Default  1744.000000           time  1744.000000  24.0000  24.0000
13510  Default  3035.000000          sound  3035.000000  23.0000  23.0000
9077   Default   254.000000          metal   254.000000  22.0000  22.0000
11405  Default   423.000000           punk   423.000000  21.0000  21.0000
10807  Default  1009.000000           play  1009.000000  20.0000  20.0000
11020  Default   421.000000          power   421.000000  19.0000  19.0000
12871  Default   745.000000            set   745.000000  18.0000  18.0000
8493   Default   974.000000           long   974.000000  17.0000  17.0000
3589   Default   341.000000          death   341.000000  16.0000  16.0000
13451  Default   547.000000           solo   547.000000  15.0000  15.0000
6336   Default   482.000000          group   482.000000  14.0000  14.0000
8255   Default   636.000000          leave   636.000000  13.0000  13.0000
7779   Default   380.000000           jazz   380.000000  12.0000  12.0000
2740   Default  1427.000000           come  1427.000000  11.0000  11.0000
8443   Default   706.000000           live   706.000000  10.0000  10.0000
12244  Default   275.000000           riff   275.000000   9.0000   9.0000
14756  Default   691.000000          title   691.000000   8.0000   8.0000
16201  Default  1390.000000           work  1390.000000   7.0000   7.0000
8441   Default  1528.000000         little  1528.000000   6.0000   6.0000
8191   Default   805.000000           late   805.000000   5.0000   5.0000
9718   Default  1778.000000            new  1778.000000   4.0000   4.0000
9221   Default   604.000000         minute   604.000000   3.0000   3.0000
13157  Default   993.000000           sing   993.000000   2.0000   2.0000
11911  Default  1044.000000        release  1044.000000   1.0000   1.0000
...        ...          ...            ...          ...      ...      ...
10302  Topic10     0.011134          ozuna     0.885698   3.3151  -9.7437
8807   Topic10     0.011125         mariah     0.887485   3.3122  -9.7446
6500   Topic10     0.015969        hapless     1.955235   2.8838  -9.3831
4536   Topic10     0.019515  effectiveness     3.054084   2.6384  -9.1825
702    Topic10     0.013641    application     1.911708   2.7487  -9.5407
7327   Topic10     0.013006     inexorably     2.109327   2.6027  -9.5884
8651   Topic10     0.012780       machines     2.115559   2.5822  -9.6059
3451   Topic10     0.012657      customize     2.082909   2.5881  -9.6156
4195   Topic10     0.012318           dock     1.985056   2.6091  -9.6427
13013  Topic10     0.012253       shooting     1.953553   2.6198  -9.6480
2166   Topic10     0.012601        cassidy     3.531034   2.0558  -9.6200
9545   Topic10     0.012323       mutilate     2.860451   2.2442  -9.6423
1938 

# Keywords for topics clustered by Latent Semantic Indexing

In [34]:

print("NMF Model:")
selected_topics(nmf, vectorizer)

NMF Model:
Topic 0:
[('song', 12.045126714128928), ('sing', 2.7668427715497192), ('feel', 1.9927954047184695), ('guitar', 1.8158763676492202), ('voice', 1.7611901958360865), ('sound', 1.6701251905732986), ('pop', 1.601385231029172), ('record', 1.4171752901040968), ('new', 1.317555483438673), ('way', 1.2790834866524643)]
Topic 1:
[('rap', 8.452630976833063), ('rapper', 1.9638600982438903), ('beat', 1.8487413128937527), ('song', 1.7839056248531857), ('little', 1.6575370044908275), ('verse', 1.471483920307999), ('way', 1.2623396399458995), ('money', 1.2101833342620891), ('year', 1.2016331528679565), ('come', 1.1932117125196455)]
Topic 2:
[('music', 6.229272773106411), ('sound', 5.727819235879542), ('track', 3.434004468997427), ('feel', 1.648644006260928), ('drum', 1.6171531952561642), ('way', 1.274455500884793), ('beat', 1.223366443525251), ('work', 1.2123479670261788), ('electronic', 1.1959374320968088), ('dance', 1.1920527160942214)]
Topic 3:
[('band', 10.881339418072688), ('rock', 4.12

In [35]:
# Keywords for topics clustered by Non-Negative Matrix Factorization
print("LSI Model:")
selected_topics(lsi, vectorizer)

LSI Model:
Topic 0:
[('album', 0.3433163663461799), ('song', 0.32830556732907934), ('sound', 0.22681263320955883), ('music', 0.21242554193834745), ('record', 0.19007254500971088), ('feel', 0.16886059527388017), ('band', 0.15706821874433816), ('time', 0.14507510893578549), ('new', 0.1398824920532345), ('track', 0.13128853370388852)]
Topic 1:
[('rap', 0.5278640445444991), ('hip', 0.15257542824346967), ('hop', 0.14884871629462038), ('beat', 0.12471618409461399), ('rapper', 0.1227821450144202), ('verse', 0.09419432346365915), ('producer', 0.07980466885698329), ('black', 0.07795544109998377), ('sample', 0.07753727460387408), ('money', 0.07321480001527703)]
Topic 2:
[('song', 0.47744322692196817), ('band', 0.15005119971438852), ('cash', 0.14798284381717602), ('love', 0.13812223987651182), ('sing', 0.12940416032400306), ('write', 0.10999804395864592), ('rock', 0.09640967745209782), ('man', 0.09310569674433716), ('woman', 0.08840935393594401), ('rap', 0.08345394956978798)]
Topic 3:
[('song', 0

# Bigram Tokenizer for LDA

In [36]:
def spacy_bigram_tokenizer(phrase):
    doc = parser(phrase) # create spacy object
    token_not_noun = []
    notnoun_noun_list = []
    noun = ""

    for item in doc:
        if item.pos_ != "NOUN": # separate nouns and not nouns
            token_not_noun.append(item.text)
        if item.pos_ == "NOUN":
            noun = item.text
        
        for notnoun in token_not_noun:
            notnoun_noun_list.append(notnoun + " " + noun)

    return " ".join([i for i in notnoun_noun_list])

In [37]:
bivectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, ngram_range=(1,2))
bigram_vectorized = bivectorizer.fit_transform(data18["processed_content"])

In [38]:
bi_lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_bi_lda = bi_lda.fit_transform(bigram_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [39]:
print("Bi-LDA Model:")
selected_topics(bi_lda, bivectorizer)

Bi-LDA Model:
Topic 0:
[('album', 0.15237365510738302), ('song', 0.14070792487186456), ('track', 0.1407027126073303), ('sound', 0.13491200538877515), ('record', 0.12740229130678096), ('band', 0.12639845245234385), ('new', 0.12383004914293266), ('work', 0.12218633717998503), ('year', 0.12121197019016479), ('music', 0.12096066078236477)]
Topic 1:
[('dylan', 78.04836724501767), ('morrison', 45.05322541874967), ('earl', 40.118646632728634), ('van', 35.09652479198229), ('warner bros', 3.8764482126300783), ('bros', 3.4811257406460068), ('warner', 2.5644096417659443), ('blues', 2.53982989613759), ('casual fan', 2.53415383354108), ('1969', 2.452414229570408)]
Topic 2:
[('ski', 13.848356437444123), ('ono', 12.78500139844079), ('mask', 0.8345933065607322), ('freakouts', 0.725997285017995), ('decade release', 0.32741943780825156), ('liberation', 0.19333909914469075), ('cusp', 0.1435967861004383), ('song', 0.14317134932639952), ('album', 0.1397787998592311), ('black people', 0.13120620478466488)]
